<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/Distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [377]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che.py
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/FlattenWrap.ipynb
!pip install importnb

In [378]:
from importnb import Notebook
with Notebook(): 
    from FlattenWrap import DotDict, Range, RangeArray, Comp, CompArray, dtox

import che
import numpy as np
from scipy.optimize import root

In [379]:
p=che.Props(['Benzene','Toluene'])

In [398]:
d=DotDict()

d.P = 150e3
d.N = 5
d.NC = p.N_comps
d.F= 100.
d.Fz = np.array([0.4,0.6])
d.FT = 370.
d.D = 0.45 * d.F
d.B = d.F - d.D

d.F_stage = d.N // 2

d.reflux_ratio = 3.


In [399]:
def boiling_points(P):
    return root(lambda T: np.diagonal(p.Pvap(T)) - P, p.Tb).x

In [400]:
d.bp = boiling_points(d.P)
bp_scaled = (d.bp - np.mean(d.bp))/2 + np.mean(d.bp)
bp_minmax = np.min(d.bp), np.max(d.bp)

# Bug in root solver: convergence failure if T_guess is exactly linearly spaced
d.T = RangeArray(np.round(np.linspace(*bp_scaled,d.N)), *bp_minmax)
d.TC = Range(bp_scaled[-1], *bp_minmax)


d.L = RangeArray(np.repeat(d.D*d.reflux_ratio,d.N), 0., 2*d.F*d.reflux_ratio)
d.V = RangeArray(np.repeat(d.D*d.reflux_ratio,d.N), 0., 2*d.F*d.reflux_ratio)

QBGuess = d.D * d.reflux_ratio* np.mean(p.HvapNB)
d.QB = Range(QBGuess, 0., 2*QBGuess )
d.QC = Range(QBGuess, 0., 2*QBGuess)

d.Lx = CompArray(np.tile(d.Fz,(d.N,1)))
d.Vy = CompArray(np.tile(d.Fz,(d.N,1)))



In [401]:
wrap, x, d2, xtod, xtodunk  = dtox(d)

In [402]:
r = DotDict()

r.MB = np.zeros((d.N,d.NC))
zeros = np.zeros((1,p.N_comps))
def eqs(d):
    r.EQUIL = d.Lx* p.Pvap(d.T) - d.Vy*d.P


    r.Lin = np.r_[(d.L[0]*d.Vy[0])[None,:], d.L[1:,None]*d.Lx[:-1]]

    r.Lout = np.r_[d.L[1:,None]*d.Lx[:-1], (d.B*d.Lx[-1])[None,:]]

    r.Vin = np.r_[d.V[1:,None]*d.Vy[1:], zeros]
    r.Vout = d.V[:,None]*d.Vy

    r.MB = r.Lin - r.Lout + r.Vin - r.Vout

    r.MB[d.F_stage-1] += d.F*d.Fz

    r.TCBP = np.atleast_1d(np.sum(d.Vy[0]*p.Pvap(d.TC)) - d.P)

    r.T = np.insert(d.T,0,d.TC)
    r.EB = np.array(p.Hl(r.Lin, r.T[:-1]) + p.Hv(r.Vin, r.T[1:]) - p.Hl(r.Lout, r.T[1:]) - p.Hv(r.Vout, r.T[1:]))
    r.EB[d.F_stage-1] += np.array(p.Hl(d.F*d.Fz, d.FT))
    r.EB[-1]+=d.QB

    r.CONDENSER = np.atleast_1d(p.Hv(r.Vout[0],r.T[1]) - p.Hl(r.Vout[0], r.T[0]) - d.QC)

    r.RF = np.atleast_1d((d.V[0]-d.D)/d.D) - d.reflux_ratio
    return np.concatenate([np.ravel(r.EQUIL), np.ravel(r.MB), np.ravel(r.EB), r.TCBP, r.RF, r.CONDENSER]), r


In [403]:
eqs_wrapped = wrap(eqs)

In [404]:
eqs(d2)

(array([ 1.36290226e+04, -4.44234253e+04,  2.18992794e+04, -3.86459466e+04,
         3.08630714e+04, -3.23037764e+04,  4.05572950e+04, -2.53594243e+04,
         5.10195077e+04, -1.77742918e+04,  0.00000000e+00,  0.00000000e+00,
         4.00000000e+01,  6.00000000e+01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -2.20000000e+01, -3.30000000e+01,
         3.82691382e+05,  3.74087373e+06, -9.34225958e+04, -9.41945339e+04,
        -2.29170907e+06,  3.47128435e+04, -1.00000000e+00, -3.52174951e+05]),
 {'CONDENSER': array([-352174.95098559]),
  'EB': array([  382691.38171138,  3740873.73361537,   -93422.59583328,
           -94194.53390304, -2291709.07109609]),
  'EQUIL': DeviceArray([[ 13629.02255472, -44423.42527566],
               [ 21899.27940849, -38645.94657908],
               [ 30863.07144633, -32303.77635102],
               [ 40557.29499375, -25359.42434725],
               [ 51019.50772983, -17774.29177916]], dtype=float64),
  'Lin': array([[54., 

In [405]:
sol=root(eqs_wrapped, x)
print(sol)
solx=sol.x

    fjac: array([[-6.01332694e-02, -3.97428522e-02,  5.10086058e-05,
        -4.78596845e-05, -2.08136515e-05,  2.05112203e-05,
        -5.17569614e-04,  2.33144816e-04,  3.92109015e-03,
        -1.92741836e-03, -9.66352006e-07,  7.24527243e-07,
         3.69823435e-07,  1.20385452e-07,  1.10811932e-06,
        -1.06581042e-06, -1.22082436e-06,  7.91104224e-07,
        -5.80071049e-07,  7.63194092e-07,  6.29446400e-01,
        -6.27015604e-01,  3.01479254e-02, -4.29217241e-02,
        -2.67204679e-02,  6.05222022e-04, -9.79909935e-10,
        -4.49408743e-01],
       [-3.11380283e-02, -2.07938655e-02, -7.82739215e-02,
        -5.55132491e-02, -2.55791930e-05,  2.51357229e-05,
        -2.86446248e-04,  1.42339927e-04,  2.52809081e-03,
        -1.19056919e-03, -3.05306230e-07,  1.62971630e-07,
        -1.77523329e-06,  2.07161170e-06,  2.68875741e-06,
        -2.54987084e-06, -1.27182872e-06,  1.04187576e-06,
        -2.72635850e-07,  2.38177470e-07,  3.31673316e-01,
         4.72333376e

In [406]:
dunk=xtodunk(solx)
dunk

{'L': array([135.        , 134.95946011, 242.17280712, 243.49778797,
        245.35629195]), 'Lx': array([[0.56855122, 0.43144878],
        [0.40492474, 0.59507526],
        [0.29401176, 0.70598824],
        [0.18860948, 0.81139052],
        [0.10574336, 0.89425664]]), 'QB': array(6212341.76104946), 'QC': array(5626081.43557023), 'T': array([377.18852208, 382.19823546, 386.00709654, 389.99483922,
        393.42068932]), 'TC': array(372.08931397), 'V': array([180.        , 179.95946014, 187.17280724, 188.49778781,
        190.35629206]), 'Vy': array([[0.759647  , 0.240353  ],
        [0.61633592, 0.38366408],
        [0.49283802, 0.50716198],
        [0.34894483, 0.65105517],
        [0.21255214, 0.78744786]])}

In [407]:
eqs(xtod(sol.x,d2))

(array([ 3.06783244e-04,  5.76696257e-05, -5.07393270e-06, -1.23193022e-05,
         5.53163409e-06, -5.42977068e-06, -3.08262519e-04,  1.06481850e-04,
        -4.24645037e-04, -7.35530921e-06,  1.05699883e-07, -7.19227984e-08,
         1.07265095e-07, -1.93966088e-08, -4.86166144e-07,  2.05659603e-07,
         3.45961581e-07, -7.87216266e-08,  6.71457201e-09, -1.15812810e-07,
         8.83688964e-03,  8.16134876e-03, -2.38573253e-02,  1.27986576e-02,
        -6.81716949e-04,  1.13502762e-03,  1.59845470e-11,  2.59395968e-03]),
 {'CONDENSER': array([0.00259396]),
  'EB': array([ 0.00883689,  0.00816135, -0.02385733,  0.01279866, -0.00068172]),
  'EQUIL': DeviceArray([[ 3.06783244e-04,  5.76696257e-05],
               [-5.07393270e-06, -1.23193022e-05],
               [ 5.53163409e-06, -5.42977068e-06],
               [-3.08262519e-04,  1.06481850e-04],
               [-4.24645037e-04, -7.35530921e-06]], dtype=float64),
  'Lin': array([[102.55234491,  32.44765509],
         [ 76.7313651